In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported!')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported!


In [4]:
# Access the data
!wget -q -O 'sanfran_data.json' https://cocl.us/sanfran_geojson
print('Data downloaded!')

Data downloaded!


In [6]:
df_data_2.shape

(155075, 18)

In [7]:
sf_data = df_data_2[['Incident_Date', 'Incident_Day', 'Incident','Resolution',
                 'Police_District', 'Neighborhood','Latitude', 'Longitude']]

sf_data[0:10]

,Incident_Date,Incident_Day,Incident,Resolution,Police_District,Neighborhood,Latitude,Longitude
0,12/31/18,Monday,Battery,Open or Active,Mission,Castro/Upper Market,37.760889,-122.435001
1,12/31/18,Monday,Battery,Cite or Arrest Adult,Tenderloin,Tenderloin,37.784453,-122.408493
2,12/31/18,Monday,"Alcohol, Under Influence Of In Public Place",Cite or Arrest Adult,Tenderloin,Tenderloin,37.784453,-122.408493
3,12/31/18,Monday,"Theft, Other Property, $50-$200",Open or Active,Southern,Financial District/South Beach,37.786852,-122.401262
4,12/31/18,Monday,"Robbery, Chain Store, W/ Other Weapon",Cite or Arrest Adult,Mission,Castro/Upper Market,37.761021,-122.432794
5,12/31/18,Monday,"Assault, Aggravated, W/ Other Weapon",Cite or Arrest Adult,Mission,Castro/Upper Market,37.761021,-122.432794
6,12/31/18,Monday,Found Person,Open or Active,Bayview,Bayview Hunters Point,37.730936,-122.373890
7,12/31/18,Monday,"Theft, From Locked Vehicle, >$950",Open or Active,Mission,Mission,37.757035,-122.418901
8,12/31/18,Monday,Lost Property,Open or Active,Mission,Mission,37.764646,-122.426382
9,12/31/18,Monday,Lost Property,Open or Active,Mission,Mission,37.755295,-122.420964


In [8]:
sf_data.shape

(155075, 8)

In [9]:
# Drop rows with missing lat long values
sf_data_2 = sf_data.dropna(subset=['Latitude', 'Longitude'])

In [10]:
sf_data_2.shape

(147215, 8)

In [11]:
# get the first 1000 crimes in the df_incidents dataframe
limit = 1000
sf_data_2 = sf_data_2.iloc[0:limit, :]

In [12]:
sf_data_2.shape

(1000, 8)

In [13]:
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

In [14]:
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of San Francisco
sanfran_map

In [17]:
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 1000 crimes and add each to the map
for lat, lng, label in zip(sf_data_2.Latitude, sf_data_2.Longitude, sf_data_2.Incident):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sanfran_map)

# add incidents to map
sanfran_map.add_child(incidents)
    
# show map
sanfran_map

In [16]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label in zip(sf_data_2.Latitude, sf_data_2.Longitude, sf_data_2.Incident):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sanfran_map